#### Necessary imports

In [2]:
# For necessary data processing and calculations
import numpy as np
import pandas as pd

# For reading and writing files
import json
import pickle
import sys
import os
import glob
import shutil
import io
from io import BytesIO
from pathlib import Path
import hashlib

# For image processing
from PIL import Image, UnidentifiedImageError
import face_recognition
import dlib
import cv2
from insightface.app import FaceAnalysis

# For face-embedding calculation
from keras_facenet import FaceNet as FN

# For machine learning
import tensorflow as tf

# For web scraping
import html
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse

# For tracking programming progress
from tqdm.notebook import tqdm
import time

/Users/wiesruyters/miniconda3/envs/multimodalmedia/lib/python3.12/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
pd.set_option('max_colwidth', None)

# NL data collection for train/test image scraping
This notebook contains the scraping method that obtains image references from three different search engines

##### Set-up web driver

In [4]:
wd = webdriver.Firefox()
wd.quit()

### Define methods 

In [5]:
def get_image_links_ddg(search_query, num_results, headless=True):
    """
    DuckDuckGo image scraper
    """
    ddg_search_url = f'https://duckduckgo.com/?q={search_query}&iax=images&ia=images'
    
    options = webdriver.SafariOptions()
    if headless:
        options.add_argument('--headless')
    
    driver = webdriver.Safari(options=options)
    
    try:
        print(f"Fetching DuckDuckGo results for: {search_query}")
        driver.get(ddg_search_url)
        time.sleep(2) 
        
        # Scroll
        last_height = driver.execute_script("return document.body.scrollHeight")
        scroll_attempts = 0
        
        while scroll_attempts < 5:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            # Check if bottom is reached or enough image urls are obtained
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
            # Check for images
            image_elements = driver.find_elements(By.CSS_SELECTOR, 'img[data-src], img[src*="external-content.duckduckgo.com"]')
            if len(image_elements) >= num_results:
                break
            scroll_attempts += 1
        
        # Find all image elements
        image_elements = driver.find_elements(By.CSS_SELECTOR, 'img[data-src], img[src*="external-content.duckduckgo.com"]')
        
        # Extract both src and data-src attr
        links = []
        for img in image_elements[:num_results]:
            src = img.get_attribute('src') or img.get_attribute('data-src')
            if src:
                if src.startswith('//'):
                    src = 'https:' + src
                links.append(src)
        
        politician_array = [search_query] * len(links)
        
        print(f"Found {len(links)} images for {search_query}")
        return politician_array, links

    except Exception as e:
        print(f"An error occurred while scraping DuckDuckGo for {search_query}: {str(e)}")
        return [search_query], []

    finally:
        driver.quit()

In [6]:
def get_image_links_bing(search_query, num_results, max_retries=3, headless=True):
    """
    Bing image scraper 
    """
    bing_search_url = f'https://www.bing.com/images/search?q={search_query.replace(" ", "+")}'

    options = webdriver.SafariOptions()
    if headless:
        options.add_argument('--headless')
    
    for attempt in range(max_retries):
        driver = None
        try:
            driver = webdriver.Safari(options=options)
            driver.get(bing_search_url)
            print(f"Attempt {attempt + 1}/{max_retries}: Fetching Bing results for '{search_query}'")
            
            time.sleep(2)
            
            # Scroll
            last_height = driver.execute_script("return document.body.scrollHeight")
            images_collected = 0
            scroll_attempts = 0
            
            while scroll_attempts < 5 and images_collected < num_results:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                
                # Check if bottom is reached or enough image urls are obtained
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
                
                # Find all image elements
                image_elements = driver.find_elements(
                    By.CSS_SELECTOR, 
                    'img.mimg:not([src^="data:"])'
                )
                images_collected = len(image_elements)
                scroll_attempts += 1
            
            # Extract image URLs
            image_elements = driver.find_elements(
                By.CSS_SELECTOR, 
                'img.mimg[src]:not([src^="data:"])'
            )
            
            links = []
            for img in image_elements[:num_results]:
                src = img.get_attribute('src')
                if src and not src.startswith('data:'):
                    # Cover for relative URLs
                    if src.startswith('/'):
                        src = f'https://www.bing.com{src}'
                    links.append(src)
            
            if not links:
                print(f"No image links found for '{search_query}' (attempt {attempt + 1})")
                if attempt < max_retries - 1:
                    time.sleep(5 * (attempt + 1))
                    continue
                return [search_query], []
            
            politician_array = [search_query] * len(links)
            print(f"Successfully collected {len(links)} images for '{search_query}'")
            return politician_array, links
            
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for '{search_query}': {str(e)}")
            if attempt < max_retries - 1:
                wait_time = 5 * (attempt + 1)
                print(f"Waiting {wait_time} seconds before retry...")
                time.sleep(wait_time)
            
        finally:
            if driver:
                driver.quit()
    
    return [search_query], []

In [7]:
def get_image_links_ecosia(search_query, num_results, max_retries=3, headless=True):
    """
    Ecosia image scraper
    """
    ecosia_search_url = f'https://www.ecosia.org/images?q={search_query.replace(" ", "+")}'
    
    options = webdriver.SafariOptions()
    if headless:
        options.add_argument('--headless')
    
    for attempt in range(max_retries):
        driver = None
        try:
            driver = webdriver.Safari(options=options)
            print(f"Attempt {attempt + 1}/{max_retries}: Fetching Ecosia results for '{search_query}'")
            
            driver.set_page_load_timeout(30)
            driver.get(ecosia_search_url)
            
            # Wait
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".image-result__link-wrapper"))
            )
            
            # Scroll
            last_height = driver.execute_script("return document.body.scrollHeight")
            images_collected = 0
            scroll_attempts = 0
            
            while scroll_attempts < 5 and images_collected < num_results:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                
                # Check if bottom is reached
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
                
                # Count the number of images collected
                image_elements = driver.find_elements(By.CSS_SELECTOR, '.image-result__link-wrapper')
                images_collected = len(image_elements)
                scroll_attempts += 1
            
            # Extract both href and src attributes
            image_wrappers = driver.find_elements(
                By.CSS_SELECTOR, 
                '.image-result__link-wrapper'
            )
            
            links = []
            for wrapper in image_wrappers[:num_results]:
                try:
                    # First: href
                    link = wrapper.find_element(By.CSS_SELECTOR, 'a.image-result__link')
                    href = link.get_attribute('href')
                    
                    # Fallback: src
                    if not href:
                        img = wrapper.find_element(By.CSS_SELECTOR, 'img.image-result__image')
                        href = img.get_attribute('src')
                    
                    if href:
                        links.append(href)
                except Exception as e:
                    print(f"Warning: Couldn't extract link from one result: {str(e)}")
                    continue
            
            if not links:
                print(f"No image links found for '{search_query}' (attempt {attempt + 1})")
                if attempt < max_retries - 1:
                    time.sleep(5 * (attempt + 1))
                    continue
                return [search_query], []
            
            politician_array = [search_query] * len(links)
            print(f"Successfully collected {len(links)} images for '{search_query}'")
            return politician_array, links
            
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for '{search_query}': {str(e)}")
            if attempt < max_retries - 1:
                wait_time = 5 * (attempt + 1)
                print(f"Waiting {wait_time} seconds before retry...")
                time.sleep(wait_time)
            
        finally:
            if driver:
                driver.quit()
    
    return [search_query], []

### Create database with image links
Create a list of politician names and set a limit on the number of image links to be extracted. <br>
For the NL, this list is based on: https://nl.wikipedia.org/wiki/Tweede_Kamerverkiezingen_2023 & https://app.nos.nl/nieuws/tk2023/ 

In [8]:
nl_politicians_23 = [
    'Geert Wilders',            # PVV
    'Frans Timmermans',         # GL-PvdA
    'Dilan Yesilgöz',           # VVD
    'Pieter Omtzigt',           # NSC
    'Rob Jetten',               # D66
    'Caroline van der Plas',    # BBB
    'Henri Bontenbal',          # CDA
    'Lilian Marijnissen',       # SP
    'Thierry Baudet',           # FVD
    'Esther Ouwehand',          # PvdD
    'Mirjam Bikker',            # CU
    'Chris Stoffer',            # SGP
    'Stephan van Baarle',       # DENK
    'Laurens Dassen',           # Volt
    'Joost Eerdmans',           # JA21
    'Edson Olf',                # Bij1
    'Wybren van Haga']          # BvNL

num_results = 125
image_link_list = []
politician_list = []

#### DuckDuckGo image links

In [9]:
for i, p in enumerate(nl_politicians_23):
    start_time = time.time()
    
    politician_array_ddg, image_links_ddg = get_image_links_ddg(p, num_results, headless=False)
    
    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)

    print(f'Completed the extraction of {len(image_links_ddg)} for politician {p} in {int(minutes)} minutes and {seconds:.2f} seconds')
    image_link_list.extend(image_links_ddg)
    politician_list.extend(politician_array_ddg)

Fetching DuckDuckGo results for: Geert Wilders
Found 125 images for Geert Wilders
Completed the extraction of 125 for politician Geert Wilders in 0 minutes and 6.04 seconds
Fetching DuckDuckGo results for: Frans Timmermans
Found 125 images for Frans Timmermans
Completed the extraction of 125 for politician Frans Timmermans in 0 minutes and 5.74 seconds
Fetching DuckDuckGo results for: Dilan Yesilgöz
Found 125 images for Dilan Yesilgöz
Completed the extraction of 125 for politician Dilan Yesilgöz in 0 minutes and 5.84 seconds
Fetching DuckDuckGo results for: Pieter Omtzigt
Found 125 images for Pieter Omtzigt
Completed the extraction of 125 for politician Pieter Omtzigt in 0 minutes and 5.68 seconds
Fetching DuckDuckGo results for: Rob Jetten
Found 125 images for Rob Jetten
Completed the extraction of 125 for politician Rob Jetten in 0 minutes and 5.81 seconds
Fetching DuckDuckGo results for: Caroline van der Plas
Found 125 images for Caroline van der Plas
Completed the extraction of 125

In [10]:
print(len(politician_list), len(image_link_list))

2125 2125


In [11]:
df_ddg = pd.DataFrame({'politician': politician_list, 'img_link': image_link_list, 'engine': 'ddg'})
df_ddg['img_link'].tail()

2120    https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse4.mm.bing.net%2Fth%2Fid%2FOIP.IDFGPCT8gqF3SZ575euiyQHaEv%3Fr%3D0%26pid%3DApi&f=1&ipt=aa6d3a15251b8f6af49531c06b7a59b43013d89561eabb8d3f4f9ea2ac2e79d7&ipo=images
2121    https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse3.mm.bing.net%2Fth%2Fid%2FOIP.Mw7SLsbgGTrChRe-6uZXEQHaE7%3Fr%3D0%26pid%3DApi&f=1&ipt=5957ab3897e7502e113b3762fe134afee7ff97052245baadb6e21759f95f4d7f&ipo=images
2122    https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse2.mm.bing.net%2Fth%2Fid%2FOIP.wcI1qCs2qoyOiDFaSszFlwHaEK%3Fr%3D0%26pid%3DApi&f=1&ipt=d8ced64a4bbbafe84ca34d45ef38a9096d7ec0167c9af6c98b52021a20ab16ec&ipo=images
2123    https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse3.mm.bing.net%2Fth%2Fid%2FOIP.z4LWHHqmbNR_UT_Of72N8wHaFX%3Fr%3D0%26pid%3DApi&f=1&ipt=036dcadbf5e2a0250e2b4e49aeaeaa53a9918dc45aa7c0bd9f8bf5fa7b8e3703&ipo=images
2124    https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1

#### Bing image links

In [12]:
image_link_list_bing = []
politician_list_bing = []

In [13]:
for i, p in enumerate(nl_politicians_23):
    start_time = time.time()
    
    try: 
        politician_array_bing, image_links_bing = get_image_links_bing(
            p, 
            num_results, 
            headless=False
        )
        
        elapsed_time = time.time() - start_time
        minutes, seconds = divmod(elapsed_time, 60)

        if image_links_bing:
            print(f"Success: {len(image_links_bing)} images for {p} in {int(minutes)}m {seconds:.1f}s")
            image_link_list_bing.extend(image_links_bing)
            politician_list_bing.extend(politician_array_bing)
        else:
            print(f"Warning: No images found for {politician}")
            
    except Exception as e:
        print(f"Critical error processing {p}: {str(e)}")

    print(f'Completed the extraction of {len(image_links_bing)} for politician {p} in {int(minutes)} minutes and {seconds:.2f} seconds')

Attempt 1/3: Fetching Bing results for 'Geert Wilders'
Successfully collected 125 images for 'Geert Wilders'
Success: 125 images for Geert Wilders in 0m 10.9s
Completed the extraction of 125 for politician Geert Wilders in 0 minutes and 10.90 seconds
Attempt 1/3: Fetching Bing results for 'Frans Timmermans'
Successfully collected 95 images for 'Frans Timmermans'
Success: 95 images for Frans Timmermans in 0m 12.6s
Completed the extraction of 95 for politician Frans Timmermans in 0 minutes and 12.63 seconds
Attempt 1/3: Fetching Bing results for 'Dilan Yesilgöz'
Successfully collected 116 images for 'Dilan Yesilgöz'
Success: 116 images for Dilan Yesilgöz in 0m 12.5s
Completed the extraction of 116 for politician Dilan Yesilgöz in 0 minutes and 12.52 seconds
Attempt 1/3: Fetching Bing results for 'Pieter Omtzigt'
Successfully collected 117 images for 'Pieter Omtzigt'
Success: 117 images for Pieter Omtzigt in 0m 12.6s
Completed the extraction of 117 for politician Pieter Omtzigt in 0 minut

In [14]:
print(len(image_link_list_bing), len(politician_list_bing))

1948 1948


In [15]:
df_bing = pd.DataFrame({'politician': politician_list_bing, 'img_link': image_link_list_bing, 'engine': 'bing'})
df_bing['img_link'].tail()

1943    https://th.bing.com/th/id/OIP.V8s7PdcX0UgxZ-8-bdVLzAHaDn?w=290&h=171&c=7&r=0&o=7&dpr=2&pid=1.7&rm=3
1944    https://th.bing.com/th/id/OIP.7XZKeH7-Z1twrtHTMNhdvQHaHa?w=146&h=180&c=7&r=0&o=7&dpr=2&pid=1.7&rm=3
1945    https://th.bing.com/th/id/OIP.jrs3uJKuz5tMEvqyPNy9cwHaEK?w=260&h=180&c=7&r=0&o=7&dpr=2&pid=1.7&rm=3
1946    https://th.bing.com/th/id/OIP._8dx833_KGVcSIFQ0lLzjAHaFY?w=200&h=180&c=7&r=0&o=7&dpr=2&pid=1.7&rm=3
1947    https://th.bing.com/th/id/OIP.Mdc2joBke0gNzUqq2MFu9QHaEK?w=260&h=180&c=7&r=0&o=7&dpr=2&pid=1.7&rm=3
Name: img_link, dtype: object

#### Ecosia image links

In [16]:
image_link_list_ecosia = []
politician_list_ecosia = []

In [18]:
for i, p in enumerate(nl_politicians_23):
    start_time = time.time()
    
    try:
            politician_array_ecosia, image_links_ecosia = get_image_links_ecosia(
                p, 
                num_results,
                headless=False
            )
            
            elapsed = time.time() - start_time
            mins, secs = divmod(elapsed, 60)
            
            if image_links_ecosia:
                print(f"Success: {len(image_links_ecosia)} images for {p} in {int(minutes)}m {seconds:.1f}s")
                image_link_list_ecosia.extend(image_links_ecosia)
                politician_list_ecosia.extend(politician_array_ecosia)
            else:
                print(f"Warning: No images found for {p}")
                
    except Exception as e:
        print(f"Critical error processing {p}: {str(e)}")

Attempt 1/3: Fetching Ecosia results for 'Geert Wilders'
Successfully collected 125 images for 'Geert Wilders'
Success: 125 images for Geert Wilders in 0m 10.4s
Attempt 1/3: Fetching Ecosia results for 'Frans Timmermans'
Successfully collected 125 images for 'Frans Timmermans'
Success: 125 images for Frans Timmermans in 0m 10.4s
Attempt 1/3: Fetching Ecosia results for 'Dilan Yesilgöz'
Successfully collected 125 images for 'Dilan Yesilgöz'
Success: 125 images for Dilan Yesilgöz in 0m 10.4s
Attempt 1/3: Fetching Ecosia results for 'Pieter Omtzigt'
Successfully collected 125 images for 'Pieter Omtzigt'
Success: 125 images for Pieter Omtzigt in 0m 10.4s
Attempt 1/3: Fetching Ecosia results for 'Rob Jetten'
Successfully collected 125 images for 'Rob Jetten'
Success: 125 images for Rob Jetten in 0m 10.4s
Attempt 1/3: Fetching Ecosia results for 'Caroline van der Plas'
Successfully collected 125 images for 'Caroline van der Plas'
Success: 125 images for Caroline van der Plas in 0m 10.4s
Atte

In [19]:
print(len(image_link_list_ecosia), len(politician_list_ecosia))

2125 2125


In [20]:
df_ecosia = pd.DataFrame({'politician': politician_list_ecosia, 'img_link': image_link_list_ecosia, 'engine': 'ecosia'})
df_ecosia['img_link'].tail()

2120    https://images.nieuwrechts.nl/artikel-foto/99d8c22c5772b69f318360755ec3b6f9/van-haga.jpg
2121                                   https://cdn.nos.nl/image/2021/07/02/759442/2560x1440a.jpg
2122         https://www.linda.nl/lindanl-assets/uploads/2021/05/06161326/Van-Haga-1800x1012.jpg
2123                                        https://i.ytimg.com/vi/hWgDhScdmvk/maxresdefault.jpg
2124               https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=729099682360713
Name: img_link, dtype: object

### Combine DuckDuckGo, Bing and Ecosia dataframes

In [22]:
nl_imgref_df = pd.concat([df_ecosia, df_bing, df_ddg], axis=0, ignore_index=True)
nl_imgref_df = nl_imgref_df.sort_values(by=['politician', 'engine'])
nl_imgref_df = nl_imgref_df.reset_index(drop=True)

In [24]:
nl_imgref_df.to_pickle('datasets/train_test/NL/nl_searchengine_imgref_results.pkl')
nl_imgref_df.to_csv('datasets/train_test/NL/nl_searchengine_imgref_results.csv')

### Collect training data

Insightface is applied to the full training image. Thereby, only the faces are cut-out and stored

In [25]:
app = FaceAnalysis(
    name='buffalo_l',
    providers=['CUDAExecutionProvider', 'CPUExecutionProvider']
)
app.prepare(ctx_id=0, det_size=(640, 640))

/Users/wiesruyters/miniconda3/envs/multimodalmedia/lib/python3.12/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:118: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CoreMLExecutionProvider, AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/wiesruyters/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/wiesruyters/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/wiesruyters/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/wiesruyters/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/wiesruyters/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None

In [ ]:
def generate_filename(url, p, face_index):
    url_hash = hashlib.md5(url.encode()).hexdigest()[:8]
    return f"{url_hash}_{p}_{face_index}.jpg"

def download_image(url, timeout=15):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, stream=True, timeout=timeout)
        response.raise_for_status()
        
        # Verify that the ref leads to an image
        if 'image' not in response.headers.get('Content-Type', ''):
            raise ValueError("Oops... no image found here!")
            
        image = np.asarray(bytearray(response.content), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        
        if image is None:
            raise ValueError("Failed to decode image")
            
        return image
    except Exception as e:
        print(f"Error downloading {url[:50]}...: {str(e)}")
        return None

def process_images(df, output_base):
    pbar = tqdm(df.iterrows(), total=len(df), desc="Processing training images")
    
    for idx, row in pbar:
        politician = row['politician']
        img_url = row['img_link']
     
        politician_safe = "".join(c if c.isalnum() else "_" for c in politician)
        politician_dir = os.path.join(output_base, politician_safe)
        os.makedirs(politician_dir, exist_ok=True)
        
        # Save image with retry
        img = None
        for attempt in range(3): # Attempts
            img = download_image(img_url)
            if img is not None:
                break
            time.sleep(1)
        
        if img is None:
            continue
        
        # Detect faces using the Insightface app (confidence threshold)
        faces = app.get(img)
        
        # Filter
        faces = [face for face in faces if face.det_score > 0.6]
        
        if not faces:
            continue
        
        # Process faces
        for i, face in enumerate(faces):
            # Bounding box
            bbox = face.bbox.astype(int)
            x1, y1, x2, y2 = bbox
            
            # Dynamic padding based on face size
            padding_factor = min(0.3, 100/max(x2-x1, y2-y1))  # Slightly larger pad for small faces
            h, w = y2 - y1, x2 - x1
            x1 = max(0, x1 - int(padding_factor * w))
            y1 = max(0, y1 - int(padding_factor * h))
            x2 = min(img.shape[1], x2 + int(padding_factor * w))
            y2 = min(img.shape[0], y2 + int(padding_factor * h))
            
            # Crop 
            face_crop = img[y1:y2, x1:x2]
            if face_crop.size == 0:
                continue
                
            # Generate filename and save
            face_filename = generate_filename(img_url, politician, i)
            save_path = os.path.join(politician_dir, face_filename)
            
            try:
                # Save
                cv2.imwrite(save_path, face_crop, [int(cv2.IMWRITE_JPEG_QUALITY), 90])
            except Exception as e:
                print(f"Error saving {save_path}: {str(e)}")

NOTE: this function is run only once. After it ran, and the facial images are collected, a manual correction is applied. With drag-and-drop, the image samples are put in the folders that contain the right label (here: politician name). <br>
DO NOT RUN THIS FUNCTION; the faces are already collected and corrected!

In [1]:
#process_images(, )